In [1]:
import pandas as pd
import numpy as np
import os
import PIL
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision.models as models
import matplotlib.pyplot as plt
from torchvision.io import read_image

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!unzip /content/gdrive/MyDrive/data_folder/2019_24k.zip > /dev/null

In [5]:
!unzip /content/gdrive/MyDrive/data_folder/2019_01_8k.zip > /dev/null

In [6]:
!unzip /content/gdrive/MyDrive/data_folder/2019_02_8k.zip > /dev/null

In [7]:
!unzip /content/gdrive/MyDrive/data_folder/2019_03_8k.zip > /dev/null

In [9]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
      
    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 5])    #5-classes 6-wholeset
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 7] #7-classes 8-wholeset

        if self.transform:
            image = self.transform(image)

        if self.target_transform:
            label = self.target_transform(label)
      
        return image, label

In [10]:
class SolarResNet50(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.resnet50(pretrained=False)
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.model.fc = nn.Linear(in_features = 2048, out_features = 512, bias=True)
        self.fc2 = nn.Linear(in_features = 512, out_features = 128, bias=True)
        self.fc3 = nn.Linear(in_features = 128, out_features = 64, bias=True)
        self.fc4 = nn.Linear(in_features = 64, out_features = 32, bias=True)
        self.fc5 = nn.Linear(in_features = 32, out_features = 16, bias=True)
        self.fc6 = nn.Linear(in_features = 16, out_features = 1, bias=True)

    def forward(self, x):
        x = self.model(x)
        x = F.relu(x)

        x = self.fc2(x)
        x = F.relu(x)

        x = self.fc3(x)
        x = F.relu(x)

        x = self.fc4(x)
        x = F.relu(x)

        x = self.fc5(x)
        x = F.relu(x)

        x = self.fc6(x)

        return x


In [55]:
model = SolarResNet50()
model

SolarResNet50(
  (model): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      

In [46]:
test_csv_file = '/content/gdrive/MyDrive/data_folder/2019_400_test_01.csv'        #in Use
image_folder = '/content/2019_01_8k'

test_set = CustomImageDataset(test_csv_file , image_folder, transform = transforms.Compose([
    transforms.ToPILImage(),
    #transforms.RandomHorizontalFlip(),  
    transforms.ToTensor(),  
]))

test_data_len = len(test_set)

#test_size = 400
batch_size = 400
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)

l = len(test_loader)

In [47]:
validation_csv_set ='/content/gdrive/MyDrive/data_folder/2019_400_evel_01.csv'    #Not in use
image_folder = '/content/2019_01_8k'

validation_set = CustomImageDataset(validation_csv_set , image_folder, transform = transforms.Compose([
    transforms.ToPILImage(),
    #transforms.RandomHorizontalFlip(),  
    transforms.ToTensor(),  
]))

validation_data_len = len(validation_set)


#validation_size = 400
batch_size = 400
validation_loader = DataLoader(dataset=validation_set, batch_size=batch_size, shuffle=True)

l = len(validation_loader)

In [50]:
model_load_name = 'SolarResNet50_01_8k.pth'
load_path = F"/content/gdrive/MyDrive/data_folder/{model_load_name}"

def load_checkpoint(checkpoint):
  print("=> loading checkpoint")
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [51]:
model = SolarResNet50()
optimizer = optim.Adam(model.parameters(),lr = 0.001)
load_checkpoint(torch.load(load_path))

model.eval()
MAE = nn.L1Loss()

def RMSE(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

for batch in test_loader: # Get batch
    image, label = batch

    pred = model(image) # Pass batch

    Y = torch.tensor(label)
    new_shape = (len(label), 1)
    Y = Y.view(new_shape)

    loss1 = MAE(pred,Y) # Calculate loss
    loss2 = RMSE(pred,Y)

    #print(f"Label: {label.item()}     ",f"Pred: {pred.item()}     ", f"Loss : {loss.item()}     ",f"Percentage Loss : {loss.item()/label.item()*100} %     " )
    print(loss1.item())
    print(loss2.item())

=> loading checkpoint
38.647457122802734
55.20071823256969


In [52]:
model = SolarResNet50()
optimizer = optim.Adam(model.parameters(),lr = 0.001)
load_checkpoint(torch.load(load_path))

model.eval()
MAE = nn.L1Loss()

def RMSE(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

for batch in validation_loader: # Get batch
    image, label = batch

    pred = model(image) # Pass batch

    Y = torch.tensor(label)
    new_shape = (len(label), 1)
    Y = Y.view(new_shape)

    loss1 = MAE(pred,Y) # Calculate loss
    loss2 = RMSE(pred,Y)

    #print(f"Label: {label.item()}     ",f"Pred: {pred.item()}     ", f"Loss : {loss.item()}     ",f"Percentage Loss : {loss.item()/label.item()*100} %     " )
    print(loss1.item())
    print(loss2.item())

=> loading checkpoint
36.61136245727539
53.129312008050235
